<a href="https://colab.research.google.com/github/srujayreddyv/CSUS-CSC219-MachineLeaning/blob/main/demo/demo_falcon_Guanaco_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Demo: Falcon-7B

In [ ]:
!pip install transformers
!pip install bitsandbytes
!pip install huggingface-hub
!pip install einops
!pip install accelerate
!pip install xformers

In [ ]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="1,2" # if you need to specify GPUs
import time
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import transformers
import torch


MODEL_NAME =  "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

streamer = TextStreamer(tokenizer)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model =AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForC

In [ ]:
prompt = """Q: You are a soccer coach in Premier League.   Who you will pick to play, Messi or Ronaldo?
A: """


sequences = pipeline(
    prompt,
    max_length=1024,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    streamer=streamer
)


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Q: You are a soccer coach in Premier League.   Who you will pick to play, Messi or Ronaldo?
A:  It's tough decision. Both are world-class players. However, it would depend on the type of team you're coaching and your coaching style. <|endoftext|>


In [ ]:
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Result: Q: You are a soccer coach in Premier League.   Who you will pick to play, Messi or Ronaldo?
A:  It's tough decision. Both are world-class players. However, it would depend on the type of team you're coaching and your coaching style. 


In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print_trainable_parameters(model)

trainable params: 295768960 || all params: 3608744832 || trainable%: 8.195895630450604


In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 256
generation_config.temperature = 1
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 11,
  "eos_token_id": 11,
  "max_new_tokens": 256,
  "pad_token_id": 11,
  "top_p": 0.7,
  "transformers_version": "4.30.2"
}

In [ ]:
prompt = """Q: How torch.nn.Parameter differs from torch.nn.Linear in Pytorch?  Provide examples
A: """


device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids=encoding.input_ids,
      attention_mask=encoding.attention_mask,
      generation_config=generation_config,
  )
print(tokenizer.decode(outputs[0],skip_special_tokens=True))


Q: How torch.nn.Parameter differs from torch.nn.Linear in Pytorch?  Provide examples
A: 
torch.nn.Parameter is a class in PyTorch that represents a tensor that can be modified during training. It is similar to a tensor in TensorFlow, but it is not a tensor in the traditional sense. It is a class that can be used to modify the values of a tensor during training.

On the other hand, torch.nn.Linear is a class in PyTorch that represents a linear model. It is used to learn a linear mapping between input and output. It is different from a tensor in that it is a linear model that learns a mapping between inputs and outputs.

For example, let's say we have a linear model that takes in two inputs and outputs a single output. We can use a torch.nn.Linear model to learn this mapping. Here's an example:

```python
import torch

class Linear(torch.nn.Linear):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        self.in_features = in_features
        sel

In [ ]:
prompt = """import matplotlib.pyplot as plt
"""


In [ ]:

sequences = pipeline(
    prompt,
    max_length=512,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    streamer=streamer
)

#for seq in sequences:
#    print(f"Result: {seq['generated_text']}")


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(0, 5, 100)
y = np.sin(x)

plt.plot(x, y)
plt.xlabel('X Label')
plt.ylabel('Y label')
plt.xlim(1, 5)
plt.show()
User <|endoftext|>


## Guanaco-7B-demo

A new method named QLoRA enables the fine-tuning of large language models on a single GPU. Researchers used it to train Guanaco, a chatbot that reaches 99 % of ChatGPTs performance.

Researchers at the University of Washington present QLoRA (Quantized Low Rank Adapters), a method for fine-tuning large language models. Along with QLoRA, the team releases Guanaco, a family of chatbots based on Meta's LLaMA models. The largest Guanaco variant, with 65 billion parameters, achieves above 99 percent of the performance of ChatGPT (GPT-3.5-turbo) in a benchmark run with GPT-4.

Fine-tuning large language models is one of the most important techniques for improving their performance and training desired and undesired behaviors. However, this process is extremely computationally intensive for large models such as LLaMA 65B, requiring more than 780 gigabytes of GPU RAM in such cases. While the open-source community uses various quantization methods to reduce 16-bit models to 4-bit models, significantly reducing the memory required for inference, similar methods have not been available for fine-tuning.

Guanaco is an advanced instruction-following language model built on Meta's LLaMA 7B model. Expanding upon the initial 52K dataset from the Alpaca model, an additional 534,530 entries have been incorporated, covering English, Simplified Chinese, Traditional Chinese (Taiwan), Traditional Chinese (Hong Kong), Japanese, Deutsch, and various linguistic and grammatical tasks. This wealth of data enables Guanaco to perform exceptionally well in multilingual environments.

In [ ]:
!sudo apt-get install cmake build-essential pkg-config libgoogle-perftools-dev
import sentencepiece

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
pkg-config is already the newest version (0.29.2-1ubuntu3).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.1).
The following additional packages will be installed:
  libunwind-dev
The following NEW packages will be installed:
  libgoogle-perftools-dev libunwind-dev
0 upgraded, 2 newly installed, 0 to remove and 17 not upgraded.
Need to get 2,353 kB of archives.
After this operation, 9,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libunwind-dev amd64 1.3.2-2build2.1 [1,883 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgoogle-perftools-dev amd64 2.9.1-0ubuntu3 [470 kB]
Fetched 2,353 kB in 1s (2,550 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be

In [ ]:
#TinyPixel/Llama-2-7B-bf16-sharded
#JosephusCheung/Guanaco


import torch
#import peft
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("TinyPixel/Llama-2-7B-bf16-sharded", use_fast=False)

model = AutoModelForCausalLM.from_pretrained(
    "TinyPixel/Llama-2-7B-bf16-sharded",
    load_in_8bit=False,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)


ImportError: ignored

In [ ]:
generation_config = model.generation_config

generation_config.max_new_tokens = 256
generation_config.temperature = 0.4
generation_config.top_p = 0.7
generation_config.top_k = 35
generation_config.repetition_penalty = 1.5
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id


In [ ]:
prompt = """### Input: Why the sky is blue?
### Response: """

In [ ]:

inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")


In [ ]:
with torch.no_grad():
    outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            generation_config=generation_config,
        )

print(tokenizer.decode(outputs[0],skip_special_tokens=True))

### Input: Why the sky is blue?  
### Response:  The light from sun travels through a long path before reaching us. During this journey, it passes by many different gases in our atmosphere and gets scattered along its way to earth's surface (the scattering of white lights). This process makes some colors more visible than others - red being one that we can see very well while other wavelength are not as easily seen because they get absorbed or reflected away instead!


In [ ]:
prompt = """### Input: You are a soccer coach.  Who is a better soccer player, Messi or Ronaldo?
### Response: """

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")


In [ ]:
with torch.no_grad():
    outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            generation_config=generation_config,
        )

print(tokenizer.decode(outputs[0],skip_special_tokens=True))

### Input: You are a soccer coach.  Who is a better soccer player, Messi or Ronaldo?
### Response: 1) If you think that messy plays for Barcelona and Ronaldinho does not play in the same team as him then I would say it's definitely ronnie who has more goals than anyone else on earth! But if they both played together at Real Madrid (which happens sometimes), well...then maybe we should talk about something different because there isn’t much difference between them anymore since their careers started out so close to each other back when everyone thought one day someone might become famous enough where people could compare themselves against another person without having any idea what kind of impact either had made until now though…..but wait!! What do u mean by "who"??? Are U asking me which country produces best players??? Or Which club wins most trophies ? Well let us see.........


In [ ]:
prompt = """### Input: Please compare torch.nn.Parameter with torch.nn.Linear in Pytorch.  Provide examples
### Response: """

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")


In [ ]:
with torch.no_grad():
    outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            generation_config=generation_config,
        )

print(tokenizer.decode(outputs[0],skip_special_tokens=True))

### Input: Please compare torch.nn.Parameter with torch.nn.Linear in Pytorch.  Provide examples
### Response: 1) Parameter is a class, while Linear has no parent classes;  2) The parameter of the linear layer can be set to None and it will not affect its performance (the same as setting weight=None);   3)...
